In [44]:
import pandas as pd
import numpy as np
from operator import itemgetter 

## Identify Overlapping Peaks of Two Peak Tables

In [2]:
def ppm_calculation(x, y):
    return abs(x-y)*10e6/y

In [3]:
def nearest_value(lst, K):
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

In [79]:
def MS1_match(mz, rt, mz_list, rt_list, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
#    print("enter match")
    df = pd.DataFrame(list(zip(mz_list, rt_list)),
               columns =[col_name_mz, col_name_rt])
    if len(df[(ppm_calculation(mz, df[col_name_mz])<=ppm_threshold) & (abs(df[col_name_rt] - rt) < rt_tolerance)])>0:
        return True
    else:
        return False

In [80]:
def overlapping_peaks(t_1, t_2, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
    flag = 0
    flagged_rows = []
    total = t_2.shape[0]
    if type(t_1[col_name_mz].iloc[0]) == np.float64:
        no_string = 1
    else:
        no_string = 0      
    mz_list = []
    rt_list = []
    if no_string == 1:
        mz_list = list(t_1[col_name_mz])
        rt_list = list(t_1[col_name_rt])
    else:
        for mz_element in t_1[col_name_mz].str.split("_"):
#            print(mz)
            mz_list += [float(x) for x in mz_element]
        for rt_element in t_1[col_name_rt].str.split("_"):
            rt_list += [float(x) for x in rt_element]
    for i in range(total):
        print(str(i*100/total) + "%", end="\r")
        second_flag = 0
        if (type(t_2[col_name_mz].iloc[i]) == np.float64) or ("_" not in t_2[col_name_mz].iloc[i]):
#            print(mz_list)
#            break
            mz = float(t_2[col_name_mz].iloc[i])
            rt = float(t_2[col_name_rt].iloc[i])
#            nearest_ids = [p for p, x in enumerate(mz_list) if x == nearest_value(mz_list, mz)] # mz_list.index(nearest_value(mz_list, mz))
            if MS1_match(mz, rt, mz_list, rt_list, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
                continue
            else:
                flag = 1
        else:
            print("mz value is a string")
            for j in range(len(t_2[col_name_mz].iloc[i].split("_"))):
                mz = float(t_2[col_name_mz].iloc[i].split("_")[j])
                rt = float(t_2[col_name_rt].iloc[i].split("_")[j])
                nearest_id = mz_list.index(nearest_value(mz_list, mz))
                if MS1_match(mz, rt, mz_list, rt_list, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
                    second_flag = 1
                    continue
            if second_flag == 0:
                flag = 1
        if flag == 1:
            flagged_rows.append(i)
#            print(mz)
#            print(mz_list[nearest_id])
#            print("------------------")
#            print(rt)
#            print(rt_list[nearest_id])
    return t_2.drop(labels=flagged_rows, axis=0)

In [54]:
def quantification(t_1, t_2, t_merged, col_name_annotation_status, desired_identification_status, col_name_metabolite_name):
    n_rows_t_1 = t_1.shape[0]
    n_rows_t_2 = t_2.shape[0]
    n_rows_t_merged = t_merged.shape[0]
    
    n_annotated_rows_t_1 = t_1[t_1[col_name_annotation_status] == desired_identification_status].shape[0]
    n_annotated_rows_t_2 = t_2[t_2[col_name_annotation_status] == desired_identification_status].shape[0]
    n_annotated_rows_t_merged = t_merged[t_merged[col_name_annotation_status] == desired_identification_status].shape[0]
    
    tpr_t_1 = round(n_annotated_rows_t_1/n_rows_t_1, 4)
    tpr_t_2 = round(n_annotated_rows_t_2/n_rows_t_2, 4)
    tpr_t_merged = round(n_annotated_rows_t_merged/n_rows_t_merged, 4)
    
    n_deduplicated_metabolites_t_1 = t_1[t_1[col_name_annotation_status] == desired_identification_status].drop_duplicates(subset = col_name_metabolite_name).shape[0]
    n_deduplicated_metabolites_t_2 = t_2[t_2[col_name_annotation_status] == desired_identification_status].drop_duplicates(subset = col_name_metabolite_name).shape[0]
    n_deduplicated_metabolites_t_merged = t_merged[t_merged[col_name_annotation_status] == desired_identification_status].drop_duplicates(subset = col_name_metabolite_name).shape[0]
    
    d = {
        'Peak Tables': ['Peak Table 1', 'Peak Table 2', 'Merged Peak Table'],
        'Total Number of Peaks': [n_rows_t_1, n_rows_t_2, n_rows_t_merged],
        'Number of True Peaks': [n_annotated_rows_t_1, n_annotated_rows_t_2, n_annotated_rows_t_merged],
        'True Positive Rate': [tpr_t_1, tpr_t_2, tpr_t_merged],
        'Number of Distinct Metabolites': [n_deduplicated_metabolites_t_1, n_deduplicated_metabolites_t_2, n_deduplicated_metabolites_t_merged]
    }
    
    df = pd.DataFrame(data=d)
    
    return df

### Local CMD VS HPC CMD

#### MS-DIAL

In [58]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-dial/AlignResult-2023282236.txt", delimiter="\t", header=4)

In [59]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-dial/AlignResult-2023291844.txt", delimiter="\t", header=4)

In [60]:
len(table_1)

25952

In [61]:
table_1.head()

,Alignment ID,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,...,SB3,SA2,SB4,SB5,SA1,SB2,SA3,SA4,SB1,SA5
0,0,35.967,100.01428,Unknown,[M+NH4]2+,adduct linked to 2205_[M+H]+,0.9,False,NaN,NaN,...,1271661.10,942749.0,1253242.1,1348959.90,765282.90,1239561.40,1229463.00,880391.20,907290.06,1132224.90
1,1,0.500,100.01443,Unknown,[M+H]2+,adduct linked to 3481_[M+Na]+,1.0,False,NaN,NaN,...,16130891.00,13925881.0,17702120.0,20029132.00,8001129.00,15492309.00,16331838.00,16200029.00,6891007.50,18298616.00
2,2,0.590,100.01444,Unknown,[M+2H]2+,NaN,1.0,False,NaN,NaN,...,9658213.00,10046386.0,12457828.0,11770822.00,8898810.00,8653832.00,10247660.00,10474542.00,12107133.00,10473924.00
3,3,31.647,100.06959,Unknown,[M+H]+,NaN,1.0,False,NaN,NaN,...,2112368.20,1108073.5,2052877.8,1913928.40,824087.44,2037453.50,1230883.00,1154022.80,2215985.00,1614246.20
4,4,1.649,100.07572,Unknown,[M+H]+,NaN,0.3,False,NaN,NaN,...,415411.38,254863.5,177510.1,169264.19,504849.25,189854.14,256529.31,188777.28,311481.53,412152.28


In [62]:
len(table_2)

25447

In [63]:
type(table_2["Average Mz"].iloc[0])==np.float64

True

In [64]:
table_merged_11 = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [65]:
len(table_merged_11)

18051

#### MS-FLO

In [232]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-flo/AlignResult-2023282236_processed.txt", delimiter="\t")

In [233]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-flo/AlignResult-2023291844_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [198]:
table_1.head()

,identifier,Alignment ID,Average Mz,Average Rt(min),duplicate_flag,adduct_flag,isotope_flag,isotope_phr,Metabolite name,Adduct type,...,SB3,SA2,SB4,SB5,SA1,SB2,SA3,SA4,SB1,SA5
0,0.03_171.99,1886,171.9936,0.031,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,3555931.0,3107714.50,3700617.8,4714087.50,4075218.80,3473687.0,3224671.2,3100795.20,4109884.80,3218963.5
1,0.03_115.11,507,115.1119,0.034,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,396968.0,441166.22,386949.2,507670.12,518073.84,416492.8,406312.0,442892.34,425433.44,421556.3
2,0.04_182.99,2206,182.9855,0.038,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,6653673.5,6435527.50,6556864.5,7956248.50,8745378.00,6570529.0,6564399.0,6728224.00,8370828.00,6689435.0
3,0.04_196.15,2621,196.1549,0.042,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,2228402.2,2745459.00,2217159.0,2885036.20,3511845.20,2311969.0,2600677.0,2587100.20,2931167.50,2586849.2
4,0.43_118.09,581,118.0864,0.427,Match #069: Possible duplicate of 0.52_118.09 ...,NaN,NaN,NaN,w/o MS2:Valine; LC-ESI-QTOF; MS2; CE,[M+H]+,...,46561192.0,47708448.00,46621160.0,44220552.00,57559904.00,47149300.0,47418344.0,46128584.00,57822920.00,47533796.0


In [199]:
table_2.head()

,identifier,Alignment ID,Average Mz,Average Rt(min),duplicate_flag,adduct_flag,isotope_flag,isotope_phr,Metabolite name,Adduct type,...,SA1,SA2,SA3,SA4,SA5,SB1,SB2,SB3,SB4,SB5
0,0.03_171.99,1848,171.9936,0.031,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,4075218.80,3107714.5,3224671.20,3100795.20,3218963.50,4109884.80,3473687.00,3555931.00,3700617.80,4714087.50
1,0.04_182.99,2151,182.9855,0.038,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,8745378.00,6435527.5,6564399.00,6728224.00,6689435.00,8370828.00,6570529.00,6653673.50,6556864.50,7956248.50
2,0.04_196.15,2554,196.1549,0.042,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,3511845.20,2745459.0,2600677.00,2587100.20,2586849.20,2931167.50,2311969.00,2228402.20,2217159.00,2885036.20
3,0.41_118.44,554,118.4398,0.407,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,35294.02,30813.0,50259.05,24106.03,33674.96,31959.85,25801.86,21137.22,19737.76,18970.89
4,0.43_118.09,539,118.0864,0.427,Match #065: Possible duplicate of 0.52_118.09 ...,NaN,NaN,NaN,w/o MS2:Valine; LC-ESI-QTOF; MS2; CE,[M+H]+,...,57559904.00,47708448.0,47418344.00,46128584.00,47533796.00,57822920.00,47149300.00,46561192.00,46621160.00,44220552.00


In [235]:
table_1["Average Mz"].str.split("_")

0        [171.9936]
1        [115.1119]
2        [182.9855]
3        [196.1549]
4        [118.0864]
            ...    
25673    [157.0355]
25674    [138.0586]
25675    [157.0354]
25676    [157.0354]
25677    [157.0356]
Name: Average Mz, Length: 25678, dtype: object

In [236]:
test = []

In [237]:
for element in table_1["Average Mz"].str.split("_"):
    test += element

In [200]:
table_merged_1 = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [201]:
len(table_merged_1)

17245

### HPC NF VS Local NF

#### MS-DIAL

In [202]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_nf/ms-dial/AlignResult-2023281418.txt", delimiter="\t", header=4)

In [203]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_nf/ms-dial/AlignResult-20232101814.txt", delimiter="\t", header=4)

In [204]:
len(table_1)

25952

In [205]:
len(table_2)

25447

In [206]:
table_merged_22 = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [207]:
len(table_merged_22)

18051

#### MS-FLO

In [72]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_nf/ms-flo/AlignResult-2023281418_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [73]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_nf/ms-flo/AlignResult-20232101814_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [74]:
len(table_1)

25678

In [75]:
len(table_2)

25414

In [78]:
[float(x) for x in ["123.33", "323.21"]]

[123.33, 323.21]

In [77]:
table_1["Average Mz"].str.split("_")[0]

['171.9936']

In [212]:
table_merged_2 = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [213]:
len(table_merged_2)

17245

### HPC CMD VS HPC NF

#### MS-DIAL

In [69]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-dial/AlignResult-2023282236.txt", delimiter="\t", header=4)

In [70]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_nf/ms-dial/AlignResult-2023281418.txt", delimiter="\t", header=4)

In [71]:
len(table_1)

25952

In [72]:
len(table_2)

25952

In [59]:
mzs = list(table_1["Average Mz"])
rts = list(table_1["Average Rt(min)"])

In [60]:
merged_list = [(mzs[i], rts[i]) for i in range(0, len(mzs))]

In [61]:
merged_list[0]

(100.01428, 35.967)

In [81]:
peak_merge_33 = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [82]:
len(peak_merge_33)

25952

#### MS-FLO

In [180]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-flo/AlignResult-2023282236_processed.txt", delimiter="\t")

In [182]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_nf/ms-flo/AlignResult-2023281418_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [183]:
len(table_1)

25678

In [184]:
len(table_2)

25678

In [185]:
peak_merge = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [226]:
len(peak_merge)

23950

### Local CMD VS Local NF

#### MS-DIAL

In [267]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-dial/AlignResult-2023291844.txt", delimiter="\t", header=4)

In [268]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_nf/ms-dial/AlignResult-20232101814.txt", delimiter="\t", header=4)

In [269]:
len(table_1)

25447

In [270]:
len(table_2)

25447

In [271]:
peak_merge_44 = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

SystemError: <class 'range'> returned a result with an error set

In [225]:
len(peak_merge_44)

24928

#### MS-FLO

In [186]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-flo/AlignResult-2023291844_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [187]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_nf/ms-flo/AlignResult-20232101814_processed.txt", delimiter="\t")

In [188]:
peak_merge_4 = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [227]:
len(peak_merge_4)

23862

## Other Tests

In [175]:
table_1[table_1["Annotation tag (VS1.0)"] == "annotated by user-defined text library"].drop_duplicates(subset = "Metabolite name").shape

(660, 42)

In [173]:
table_1[table_1["Annotation tag (VS1.0)"] == "annotated by user-defined text library"]

,Alignment ID,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,...,SA1,SA2,SA3,SA4,SA5,SB1,SB2,SB3,SB4,SB5
263,263,0.593,109.07635,C6H8N2,[M+H]+,NaN,1.0,False,0.59,109.07630,...,4.606381e+07,52049700.00,48648996.00,48866940.00,50095360.00,49510468.00,4.723662e+07,4.427318e+07,48335940.00,48993324.00
265,265,0.528,109.07640,C6H8N2,[M+H]+,NaN,1.0,False,0.59,109.07630,...,2.600366e+07,25304870.00,26239746.00,25760260.00,26979930.00,27128062.00,2.614744e+07,2.504948e+07,26958778.00,26436970.00
449,449,0.581,114.12804,C7H15N,[M+H]+,NaN,1.0,False,0.58,114.12800,...,5.413586e+07,47681840.00,45654152.00,48545060.00,44667432.00,48684136.00,4.539799e+07,4.475062e+07,44759100.00,48471236.00
562,562,5.085,119.06066,C7H6N2,[M+H]+,NaN,1.0,False,5.08,119.06060,...,5.412722e+07,58578392.00,48993616.00,51911076.00,49807312.00,47893944.00,4.601670e+07,4.810860e+07,49952244.00,49673528.00
740,740,0.672,126.05515,C6H7NO2,[M+H]+,NaN,0.8,False,0.63,126.05510,...,1.217748e+07,10208069.00,10337025.00,10018535.00,9628490.00,8517147.00,9.200289e+06,9.645849e+06,9413457.00,10822839.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25021,25021,31.697,980.57556,C53H83NO14,[M+Na]+,adduct linked to 24934_[M+H]+,1.0,False,31.70,980.57220,...,4.133900e+07,47972908.00,49342812.00,47552156.00,48896308.00,50457108.00,4.722142e+07,4.351532e+07,44695528.00,45775832.00
25142,25142,4.495,1019.45032,C49H66N10O10S2,[M+H]+,ion correlated with 2281,0.5,False,4.45,1019.45007,...,2.092166e+06,1821976.80,1716858.20,1969067.80,1968747.90,791041.56,1.685933e+05,5.777482e+04,78292.47,114933.63
25143,25143,4.390,1019.45239,C49H66N10O10S2,[M+H]+,ion correlated with 1163,0.3,False,4.45,1019.45007,...,4.415434e+04,107192.33,141995.03,103186.31,101158.01,371036.10,2.422940e+05,1.903450e+06,2249673.50,2219476.80
25144,25144,4.453,1019.45251,C49H66N10O10S2,[M+H]+,NaN,0.3,False,4.45,1019.45007,...,4.510274e+05,1864471.90,660375.30,924083.56,1182922.50,2023898.00,2.069770e+06,7.085063e+05,434584.72,76172.26


In [168]:
table_merged.shape

(17245, 47)

In [178]:
summary_df = quantification(table_1, table_2, table_merged, "Annotation tag (VS1.0)", "annotated by user-defined text library", "Metabolite name")

In [179]:
summary_df

,Peak Tables,Total Number of Peaks,Number of True Peaks,True Positive Rate,Number of Distinct Metabolites
0,Peak Table 1,25447,1031,0.0405,660
1,Peak Table 2,25447,1031,0.0405,660
2,Merged Peak Table,17245,924,0.0536,629
